In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

/home/erich/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/erich/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/erich/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/erich/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.

In [3]:
# Preparing the data
# Import the data, data is stacked. This dataset contains 100000 movie ratings (movieLens datasets).
# All movieLens datasets can be found at https://grouplens.org/datasets/movielens/
ratings_df = pd.read_csv('~/Downloads/Exercises/u.data', sep='\t')
ratings_pt = ratings_df.pivot(index = 'user_id', columns ='item_id', values = 'rating').fillna(0)

# Break data into a training and test set
train_ratings_df = ratings_df[0:80000]
test_ratings_df = ratings_df[80000:100000]

user_indecies_train = [i-1 for i in train_ratings_df.user_id.values]
item_indecies_train = [i-1 for i in train_ratings_df.item_id.values]
R_known_train = train_ratings_df.rating.values

user_indecies_test = [x-1 for x in test_ratings_df.user_id.values]
item_indecies_test = [x-1 for x in test_ratings_df.item_id.values]
R_known_test = test_ratings_df.rating.values
ratings_pt

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Prepare data
R = ratings_pt
N = 943      # number of users
M = 1682     # number of items
K = 10       # number of hidden features
p = np.random.rand(N,K)
q = np.random.rand(K,M)

In [5]:
# Prediction
P = tf.Variable(p, dtype=tf.float32, name='P')
Q = tf.Variable(q, dtype=tf.float32, name='Q')
R_hat = tf.matmul(P, Q)

# For cost function, we want to compare only the known values
R_hat_flatten = tf.reshape(R_hat, [-1])
R_hat_train_known = tf.gather(R_hat_flatten, user_indecies_train * tf.shape(R_hat)[1] + 
              item_indecies_train, name='extracting_user_rate')

In [6]:
# Cost function
diff_ratings = tf.subtract(R_hat_train_known, R_known_train, name='diff_ratings')
diff_ratings_abs = tf.abs(diff_ratings, name="diff_ratings_abs")
base_cost = tf.reduce_sum(diff_ratings_abs, name="sum_abs_error")



In [7]:
# Regularization
lda = tf.constant(.001, name='lambda')
norm_sums = tf.add(tf.reduce_sum(tf.abs(P, name='user_abs'), name='user_norm'),
                   tf.reduce_sum(tf.abs(Q, name='item_abs'), name='item_norm'))
regularizer = tf.multiply(norm_sums, lda, 'regularizer')

In [8]:
# Total Cost
cost = tf.add(base_cost, regularizer)

In [9]:
# Create an optimizer to minimize the loss
optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(cost)

In [10]:
# Execute the TF session
sess = tf.Session();
init = tf.global_variables_initializer()
sess.run(init)

for i in range(500):
    sess.run(train)

In [11]:
# Accuracy for known R ratings vs predicted R_hat ratings of testset
R_hat_test_known = tf.gather(R_hat_flatten, user_indecies_test * tf.shape(R_hat)[1] + item_indecies_test, name='extracting_user_rate_test')
diff_ratings_test = tf.subtract(R_hat_test_known, R_known_test, name='diff_ratings_test')
diff_ratings_abs_test = tf.abs(diff_ratings_test, name="diff_ratings_abs_test")
sum_diff_ratings_abs_test = tf.reduce_sum(diff_ratings_abs_test, name="sum_diff_ratings_abs_test")
accuracy_test = tf.div(sum_diff_ratings_abs_test, R_known_test.size, name="accuracy_test")
print(sess.run(accuracy_test))

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
0.81553805


In [12]:
# Visualize known R ratings, to compare with R_hat.
R.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Visualize R_hat rating predictions, to compare with known R.
pred = sess.run(R_hat)
pred_df = pd.DataFrame(pred)
pred_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,4.177065,3.123865,2.568332,3.566141,3.283359,4.854988,4.679597,5.045010,5.036920,3.926876,...,3.115902,2.289635,2.933472,3.317979,3.298774,1.433172,3.216660,3.017157,2.759439,2.886139
1,4.269350,3.408203,2.800376,3.725889,3.232534,3.483443,4.085959,4.259156,4.462228,3.276952,...,2.394452,3.041770,2.927978,3.302792,2.736217,1.281388,3.282093,2.877190,3.008683,3.763845
2,3.907284,2.312826,1.258871,3.070843,2.816858,2.760060,2.526668,2.840748,3.834568,2.217962,...,1.360353,1.926891,1.853855,2.755954,1.863631,1.633914,2.446429,2.654223,2.536569,2.770681
3,5.387659,3.918978,4.057040,4.551030,4.477121,5.687111,5.309037,5.133316,5.535487,6.663755,...,3.875065,3.360760,3.624972,3.645873,3.923070,1.414596,4.332371,4.837182,3.830948,3.942286
4,3.964614,2.444060,0.359272,2.401273,3.884633,4.775637,3.253334,4.528123,3.917083,1.998634,...,2.131785,0.987617,1.206684,3.018358,3.012581,1.847365,2.807479,2.906826,1.864220,0.955193
